## Step 0 - Set variables

In [423]:
# Step 0 block 1:  install libraries

#pip install azure-storage-blob
#pip install pandas
#pip install pywin32

In [424]:
# Step 0 block 2:  import libraries

import win32com.client, datetime 
from datetime import datetime, date, timedelta  
from dateutil.parser import *
import calendar
import os
import pandas as pd
from azure.storage.blob import  BlobServiceClient, BlobClient, ContainerClient, __version__

In [425]:
# Step 0 block 3:  set the variables

vAccount = '[Mark]' # add you own account name
vStartDate = '18/09/2022' # the day you want to start syncing
vNumberofWeeks = 4 # how far in the future it should sync

In [426]:
# Step 0 block 4:  calculate the end date

specific_date = datetime.strptime(vStartDate, "%d/%m/%Y")
new_date = specific_date + timedelta(vNumberofWeeks*7)
vEndDate =new_date.date().strftime('%d/%m/%y')

print (vStartDate)
print (specific_date)
print (vEndDate)

18/09/2022
2022-09-18 00:00:00
16/10/22


## step 1 - Read how many accounts there are

In [427]:
# Step 1, block 1:  Set connection string
connect_str = "" #create connection string from Azure portal to a blob storage account
# see https://docs.microsoft.com/en-gb/azure/storage/blobs/storage-quickstart-blobs-python?tabs=environment-variable-windows for the steps
print(connect_str)

DefaultEndpointsProtocol=https;AccountName=agendaitems;AccountKey=2q4rb8eLVYNudOLm5u5wmnuMtnBA36o0d3F7OJuUKYzBHe5uMZ1qnqtUBCksKUn+AyLYhI0PXXMN+AStoQc8GQ==;EndpointSuffix=core.windows.net


In [428]:
# Step 1 block 2: download all files from blob storage

try:

    # Create the BlobServiceClient object which will be used to create a container client
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    container_name = 'agenda'
    # List containers
    container_client = blob_service_client.get_container_client(container_name)

    print("\nListing blobs...")


    # List the blobs in the container
    blob_list = container_client.list_blobs()
    blob_client = blob_service_client.get_container_client(container= container_name) 


    for blob in blob_list:
        print("\t" + blob.name)
        download_file_path = './Agenda Synchronisation/' + blob.name
        with open(download_file_path, "wb") as download_file:
            download_file.write(blob_client.download_blob(blob.name).readall())

except Exception as ex:
    print('Exception:')
    print(ex)


Listing blobs...
	[Mark].csv
Exception:
[Errno 13] Permission denied: './Agenda Synchronisation/[Mark].csv'


## Step 2 - Read all calender Items (from own agenda)

In [429]:
# Step 2, block 1 : access Outlook and get events from the calendar
Outlook = win32com.client.Dispatch('Outlook.Application')
ns = Outlook.GetNamespace('MAPI')
appts = ns.GetDefaultFolder(9).Items

In [430]:

# Step 2, block 2 : sort events by occurrence and include recurring events
appts.Sort('[Start]')
appts.IncludeRecurrences = 'True'

In [431]:
# Step 2, block 3 : block out the events that are not in the date range

restriction = "[Start] >= '" + vStartDate + "' AND [End] <= '" +vEndDate + "'"
appts = appts.Restrict(restriction)

print(restriction)
print(vStartDate)
print(vEndDate)


[Start] >= '18/09/2022' AND [End] <= '16/10/22'
18/09/2022
16/10/22


In [432]:
# Step 2, block 4 : get all accounts from that directory

#os.listdir()
agendas = os.listdir("./Agenda Synchronisation")
print(agendas)
# remove csv from name in agendas
agendas = [x.replace('.csv', '') for x in agendas]
print(agendas)

['[Anouk].csv', '[Mark].csv', '[Rabobank].csv']
['[Anouk]', '[Mark]', '[Rabobank]']


In [433]:
# Step 2, block 5 : populate array with meetings from our own agenda

apptDict = {}
item = 0
for indx, a in enumerate(appts):
     # if a part of subject is in agenda list
     if any(x in a.Subject for x in agendas):
              subject = str(a.Subject)
     else:
              subject = vAccount + ' '+ a.subject
              #organizer = str(a.Organizer)
              meetingStartDate = str(a.Start)
              meetingEndDate = str(a.end)
              StartDate = parse(meetingStartDate).date()
              EndDate = parse(meetingEndDate).date()   
              duration = str(a.duration)
              location = str(a.location)
              organiser = str(a.GetOrganizer())
              apptDict[item] = {
                     'Duration': duration,  
                     'Subject':  subject, 
                     #'StartDate': meetingStartDate.strftime('%d/%m/%Y  %H:%M:%S'),
                     'StartDate': meetingStartDate,
                     'EndDate' : EndDate.strftime('%m/%d/%Y'),
                     'Organizer' : organiser,
                     'Location' : location,
                     'Categories': vAccount
              
              }
              item = item + 1
    
   # organiser nog niet 

In [ ]:
# Step 2, block 6 : create a dataframe from the array`
# `
apt_df = pd.DataFrame.from_dict(apptDict, orient='index', columns = ['Duration', 'Organizer', 'Subject', 'StartDate', 'Location', 'Categories'])
apt_df

## Step 3 - Export calendar items to csv

In [435]:
# Step 3, block 1 : write the data frame to a csv file
filename = "./Agenda Synchronisation/" + vAccount + '.csv'
apt_df.to_csv (filename, index = True, header=True)
print (filename)

./Agenda Synchronisation/[Mark].csv


## Step 4 - Upload csv to blob

In [436]:
# Step 4, block 1 : upload the csv file to blob storage

try:    
    # Create the BlobServiceClient object which will be used to create a container client
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    
    #Upload the meeting list to the blob
    #print (upload_file_path)

    container_name = "agenda" 
    
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=vAccount + '.csv')

    print("\nUploading to Azure Storage as blob:\n\t" + vAccount)
    
    #upload the file
    with open(filename, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    

except Exception as ex:
    print('Exception:')
    print(ex)


Uploading to Azure Storage as blob:
	[Mark]


## Step 5 delete Items from other account


In [418]:
# Step 5, block 1: determine the length of the calendar
items = 0
for a in appts:
    items = items + 1
    


In [419]:
# Step 5, block 2: reverse loop the agenda, delete the events that are from another account

for a in range(items, 0, -1):
    subject_name = appts(a).subject[0:appts(a).subject.find(" ", 1) ]
    
    #print(a.subject.find(" ", 1))
    if  subject_name in agendas:
        appts(a).delete()              


## Step 6 - Import all other accounts to agenda

In [420]:
# Step 6, block 1: read all csv's and import them into the calendar

for agenda in agendas:
    if (agenda != vAccount):
        file_name =  "./Agenda Synchronisation/" + agenda + '.csv'

        #load csv into dataframe
        df = pd.read_csv(file_name, sep=',')

        #load agenda items
        outlook = win32com.client.Dispatch("Outlook.Application")
        
        for index, row in df.iterrows():
            appt = outlook.CreateItem(1)
            appt.Start = row['StartDate'].removesuffix('+00:00')
            appt.Duration = row['Duration']
            appt.Subject = row['Subject']
            appt.Location = row['Location']
            appt.Categories = row['Categories']
            appt.Sensitivity = 2
            appt.Save()        